In [2]:
import requests 
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import time 
import re 
from tqdm.notebook import tqdm
import random

import shutil #이미지저장

### get urls  많이 하면 인스타그램에서 로봇으로 판정하는듯함.
* 해결방안은 있을까?

### 통합
* 1건당 약 7초(4초 휴식), 417건 기준 30초

In [ ]:
#크롬드라이버 작동
user = 'id'
password = 'pw'
keyword = "풀빌라" 
headers = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.130 Safari/537.36'}
img_save = True
url = "https://www.instagram.com/explore/tags/{}/".format(keyword) 
# 크롬드라이버 작동 
options = Options()
options.add_argument('--start-maximized') #전체화면
options.add_argument(f'user-agent={headers}')
options.add_argument('--headless') #headless(화면안켜고)
driver = webdriver.Chrome(executable_path='./chromedriver', chrome_options=options)
driver.get(url) 
time.sleep(3) 

#로그인(id, password 입력)
ele = driver.find_element_by_css_selector('#loginForm > div > div:nth-child(1) > div > label > input')
ele.send_keys(user)
ele = driver.find_element_by_css_selector('#loginForm > div > div:nth-child(2) > div > label > input')
ele.send_keys(password)
ele = driver.find_element_by_css_selector('#loginForm > div > div:nth-child(3) > button > div')
ele.click()
time.sleep(3)
ele = driver.find_element_by_css_selector('#react-root > section > main > div > div > div > div > button') #로그인저장 창 통제
ele.click()

#url 수집
url_halfs = []
count = 1
for i in tqdm(range(0, 50)): #약 350건 
    source = driver.page_source
    soup = BeautifulSoup(source, 'html.parser')
    
    #url저장
    url_source = soup.select('#react-root > section > main > article  a')
    for url_half in url_source:
        url_halfs.append(url_half.attrs['href'])
        
    #중복제거
    url_halfs = list(set(url_halfs))
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);") #휠스크롤
    time.sleep(0.7)
    count =+ i
#     print(count)
print(f'수집된 url수:{len(url_halfs)}')

#각 게시글에서 정보추출
nonetype = type(print()) #타입비교 용도
index_dict = dict() #데이터 임시저장
if img_save == True:
    os.makedirs('./img',exist_ok=True) #이미지_저장폴더

for index, url_half in tqdm(enumerate(url_halfs[100:500])):
    #저장 초기화
    contents = dict()
    contents['dates'] = []
    contents['user_id'] = []
    contents['body'] = []
    contents['hashtags'] = []
    contents['comments'] = []
    contents['like'] = []
    contents['movie'] = []
    contents['url'] = []
    contents['image'] =[]
    real_url = f'https://www.instagram.com/{url_half}'
    driver.get(real_url)
    source = driver.page_source
    soup = BeautifulSoup(source, 'html.parser')
    
    #사람처럼 행동 random하게 스크롤 작동
    ran_cnt = random.randint(1,5)
    ran_length = random.choice(np.arange(100,200,100))
    for count in range(0,ran_cnt):
        driver.execute_script(f"window.scrollTo(0, {ran_length})")
        ran_length *= 2
        time.sleep(0.5)
    
    #dates
    try :
        dates = soup.select('div.k_Q0X.NnvRN')[0].text
    except :
        dates = ''
    
    #id
    try :
        user_id = soup.select('span.Jv7Aj.mArmR.MqpiF')[0].text
    except :
        user_id = ''
    
    #본문
    try :
        body = soup.select_one('body div.C4VMK > span').text
    except : 
        body = []
    
    #comment
    try : 
        comments = soup.select('div.C4VMK > span')
        comments = [comment.text for comment in comments]
    except:
        comments = []
        
    #해쉬태그
    try :
        hashtags = soup.find_all(attrs={'class':'xil3i'})
        hashtags = [hashtag.text for hashtag in hashtags]
    except :
        print('no hashtag')
        hashtags.append([])

    #좋아요수(동영상의 경우 view) #코드 요약 필요할듯 
    p_tag = soup.select_one('div.Nm9Fw') #사진테그
    m_tag = soup.select_one('span.vcOH2') #동영상테그
    if type(m_tag) == nonetype: #사진인경우
#         print('p_tag')
        try:
            movie=0
            like = soup.select_one('div.Nm9Fw span').text #좋아요 수         
        except :
            like = 0
    elif type(p_tag) == nonetype:#동영상인경우
#         print('m_tag')
        try :
            movie=1
            like = soup.select_one('div.vcOH2 span').text #조회수 수         
        except :
            like = 0
    #최소화해놓으면 안되고 최소화 하지 않으면 작동안함()
    random_stop = random.randint(3,5) #인간답게하려고 랜덤으로 2~4초 스탑
    time.sleep(random_stop)
    if img_save == True:
        try :
            imgurl = soup.select('div.KL4Bh')[0].img['src']
            response = requests.get(imgurl, stream = True)
            local_file = open(f'./img/{keyword}_{index}.jpg','wb') #저장임시파일
            response.raw.decode_content = True #역할이...?
            shutil.copyfileobj(response.raw, local_file)
#             print(f'{index}, 이미지저장')
            image = 1
        except Exception as ex:
#             print(f'{index}, 이미지저장실패')
#             print(ex) 
            image = 0
            pass
      
    #저장
    contents['body'].append(body)
    contents['comments'] = contents['comments'] + comments
    contents['hashtags'] = contents['hashtags'] + hashtags
    contents['dates'] = dates
    contents['user_id'] = user_id
    contents['like'] = like
    contents['movie'] = movie
    contents['url'] = real_url
    contents['image'] = image
    
    #벤방지
    time.sleep(random_stop) 
    
    #인덱스맞춰저장
    index_dict[index] = contents

driver.close()
#데이터프레임화
temp = pd.DataFrame(index_dict)
temp = temp.T
temp.to_csv(f'{keyword}_{len(temp)}.csv')

C:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:15: DeprecationWarning: use options instead of chrome_options
  from ipykernel import kernelapp as app
